In [69]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine 

#Installing dependencies
! pip install openpyxl

In [2]:
# Importing datasets

original_df_csv = pd.read_csv("cyberbullying.csv")
original_df_xlsx = pd.read_excel("1 - cyberbullying_in_social_media.xlsx") #A des problème d'encodage sur la colonne "Text", "SendenLocation", on va utilisé celle du CSV qui n'a pas le problème.

# NB : on ne prend pas le CSV complet car le nombre (notamment "Id" sont arrondie ce qui pose des problèmes de duplications)

In [3]:
#Merge CSV and XLSX

clean_df = original_df_xlsx
clean_df["Text"] = original_df_csv["Text"]
clean_df["SenderLocation"] = original_df_csv["SenderLocation"]

In [4]:
#Drop "IdRetweet"

original_df_xlsx["IsRetweet"].value_counts() #Cette colonne n'apporte aucune info, on va la supprimer.
clean_df = clean_df.drop(columns=["IsRetweet"])

In [5]:
#Cleaning "Id"

clean_df["Id"] = clean_df["Id"].astype(int)
#clean_df["Id"][clean_df.duplicated()] #On vérifie qu'il n'y ait pas de doublon

In [6]:
#Cleaning "SenderAccountYears"
clean_df["SenderAccountYears"].value_counts() # 2020 :4, 1200 : 1 -> on supprime les lignes en question

clean_df = clean_df.drop(clean_df[(clean_df["SenderAccountYears"] == 2020) | (clean_df["SenderAccountYears"] == 1200)].index) 

In [7]:
# Clean columns names
clean_df.columns = [x.replace("#","") for x in list(clean_df.columns)]

In [8]:
clean_df.set_index(keys="Id", inplace=True)

In [9]:
clean_df.to_excel("CleanDataset.xlsx")

# Benois

In [10]:
# def replace_zero_with_false(val, column_name):
#     if column_name in ['isretweet', 'isselfmentioned'] and (val == 0 or  val == 0.0 or val == 1.0):
#         return False
#     return val

# # Loop through the DataFrame columns and apply the replacement selectively
# for col in df.columns:
#     clean_df[col] = clean_df.apply(lambda x: replace_zero_with_false(x[col], col), axis=1)

# SenderLocation

In [14]:
cities_df = pd.read_csv("worldcities.csv")
cities = cities_df["city"] # On ne garde pas que les ville Turque, on veut aussi match Oslo par exemple 
cities = [city.lower() for city in cities]

In [15]:
print(cities)

['tokyo', 'jakarta', 'delhi', 'guangzhou', 'mumbai', 'manila', 'shanghai', 'são paulo', 'seoul', 'mexico city', 'cairo', 'new york', 'dhaka', 'beijing', 'kolkāta', 'bangkok', 'shenzhen', 'moscow', 'buenos aires', 'lagos', 'istanbul', 'karachi', 'bangalore', 'ho chi minh city', 'ōsaka', 'chengdu', 'tehran', 'kinshasa', 'rio de janeiro', 'chennai', 'xi’an', 'lahore', 'chongqing', 'los angeles', 'baoding', 'london', 'paris', 'linyi', 'dongguan', 'hyderābād', 'tianjin', 'lima', 'wuhan', 'nanyang', 'hangzhou', 'foshan', 'nagoya', 'taipei', 'tongshan', 'luanda', 'zhoukou', 'ganzhou', 'kuala lumpur', 'heze', 'quanzhou', 'chicago', 'nanjing', 'jining', 'hanoi', 'pune', 'fuyang', 'ahmedabad', 'johannesburg', 'bogotá', 'dar es salaam', 'shenyang', 'khartoum', 'shangqiu', 'cangzhou', 'hong kong', 'shaoyang', 'zhanjiang', 'yancheng', 'hengyang', 'riyadh', 'zhumadian', 'santiago', 'xingtai', 'chattogram', 'bijie', 'shangrao', 'zunyi', 'sūrat', 'surabaya', 'huanggang', 'maoming', 'nanchong', 'xinyan

In [33]:
# Define the replacement dictionary
replacements = {
    'Türkiye':'turkey',
    'turkiye':'turkey',
    'türkiye':'turkey',
    'turkeykocaeliizmit':'turkey',
    'istanbulturkey':'istanbul',
    'istanbulzeytinburnu':'istanbul',
    'na':np.nan,
    'NA':np.nan,
    '3406':'istanbul', #because 03406 is postal code of istanbul
    '':np.nan,
    'üsküdar': 'istanbul', #un disctict d'Istambul
    'istabul':'istanbul',
    'karşıyaka':'turkey', #un district turque
    'istanbulizmir'
    
}
# Replace values in the senderlocation column based on the replacements dictionary
clean_df['SenderLocation'] = clean_df['SenderLocation'].replace(replacements, regex=True)

In [63]:
custom_match_places = cities + ["turkey","izmir","karşıyaka"]

In [64]:
locations_not_match = [city for city in clean_df["SenderLocation"] if city not in custom_match_places and not pd.isnull(city)]

# problème de match avec "i̇"

In [55]:
pd.isnull(locations_not_match[0])

True

In [70]:
selected_features = ['Emojis', 'Punctuations', 'UpperCaseLetter', 'Letter', 'Symbols', 'Words', 'TWords',
                     'UWords', 'SlangWords', 'AvgWordLength', 'Retweets', 'Favorites', 'Medias', 'Mentions',
                     'SenderAccountYears', 'SenderFavorites', 'SenderFollowings', 'SenderFollowers', 'SenderStatues']
# Calculate descriptive statistics for selected features
descriptive_stats = clean_df[selected_features].describe()
# Print the results
print(descriptive_stats)

            Emojis  Punctuations  UpperCaseLetter       Letter      Symbols  \
count  4989.000000   4995.000000      4995.000000  4995.000000  4995.000000   
mean      0.396272      2.522122         4.260661    83.133934     0.082282   
std       1.683662      4.026303        10.660042    62.537815     0.474243   
min       0.000000      0.000000         0.000000     4.000000     0.000000   
25%       0.000000      0.000000         1.000000    35.000000     0.000000   
50%       0.000000      1.000000         1.000000    63.000000     0.000000   
75%       0.000000      4.000000         4.000000   116.000000     0.000000   
max      55.000000     84.000000       239.000000   249.000000     8.000000   

             Words       TWords       UWords   SlangWords  AvgWordLength  \
count  4995.000000  4995.000000  4995.000000  4995.000000    4825.000000   
mean     14.065666     2.682482     0.350751     0.781181       5.536995   
std      10.470609     3.634988     1.747559     0.914423   

In [74]:
# To SQL
pw = "password1999"
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string)

In [76]:
clean_df.to_sql("main",schema="datacleaning", con = engine)

4995